In [1]:
from pathlib import Path
import subprocess


def run_dcm2nii_over_bids(
    bids_root: str | Path,
    dcm2nii_cmd: str = "dcm2niix",
    sub_folder: str = "sub-",
    ses_folder: str = "ses-",
    dcm_folder: str = "DCM",
    fname: str = "T1"
) -> None:
    """
    Convert every /DCM folder in a BIDS tree to NIfTI and drop
    the output in the matching /ANAT folder.

    Layout expected
    ---------------
    <bids_root>/
        sub-XXX/
            ses-YYY/
                DCM/      <-- raw DICOM files (already organised)
                ANAT/     <-- will receive the NIfTI output
    """
    bids_root = Path(bids_root).expanduser().resolve()

    # locate every DCM directory: sub-*/ses-*/DCM
    for dcm_dir in bids_root.glob(f"{sub_folder}*/{ses_folder}*/{dcm_folder}"):
        if not dcm_dir.is_dir():
            continue

        anat_dir = dcm_dir.parent / "ANAT"
        anat_dir.mkdir(exist_ok=True)

        try:
            subprocess.run(
                [dcm2nii_cmd, "-b", "y", "-z", "y", "-o", anat_dir, str(dcm_dir)],
                check=True,
            )
        except subprocess.CalledProcessError as e:
            print(f"Error processing {dcm_dir}: {e}")
            continue

        # OPTIONAL — rename first *.nii* to sub-XXX_T1.nii.gz
        nifti_files = sorted(anat_dir.glob("*.nii*"))
        if len(nifti_files) == 1:
            subj_id = dcm_dir.parents[1].name.lstrip("sub-")
            nifti_files[0].rename(anat_dir / f"{fname}.nii.gz")


In [2]:
run_dcm2nii_over_bids('/Volumes/LaCie/BIDS', 'dcm2niix',
                      sub_folder = '')

Chris Rorden's dcm2niiX version v1.0.20230411  Clang14.0.3 ARM (64-bit MacOS)
Found 2 DICOM file(s)
Convert 1 DICOM as /Volumes/LaCie/BIDS/sub-54456298/ses-01/ANAT/DCM_Pre_Ax_T1_3D_MPRAGE_20250226071903_5 (216x256x192x1)
Convert 1 DICOM as /Volumes/LaCie/BIDS/sub-54456298/ses-01/ANAT/DCM_Post_Ax_T1_3D_MPRAGE_20250226071903_16 (216x256x192x1)
Conversion required 5.072190 seconds (4.239344 for core code).
Chris Rorden's dcm2niiX version v1.0.20230411  Clang14.0.3 ARM (64-bit MacOS)
Found 640 DICOM file(s)
Convert 320 DICOM as /Volumes/LaCie/BIDS/sub-50083831/ses-01/ANAT/DCM_8DL_Brain_Seizure_New_On_20240501191131_7 (512x512x320x1)
Convert 320 DICOM as /Volumes/LaCie/BIDS/sub-50083831/ses-01/ANAT/DCM_8DL_Brain_Seizure_New_On_20240501191131_2 (512x512x320x1)
Conversion required 25.095215 seconds (18.266657 for core code).
Chris Rorden's dcm2niiX version v1.0.20230411  Clang14.0.3 ARM (64-bit MacOS)
Found 416 DICOM file(s)
Convert 208 DICOM as /Volumes/LaCie/BIDS/sub-47163159/ses-01/ANAT/DC